In [30]:
from spacy.pipeline.spancat import DEFAULT_SPANCAT_MODEL
from spacy.pipeline import SpanCategorizer
from spacy.training import Example
from spacy.tokens import SpanGroup
import spacy
import json

In [31]:
train_data = json.load(open('./data_deep_extract/training/train_data.json','rb')) 
nlp = spacy.load("en_core_web_lg")

In [32]:
nlp=spacy.blank('en')
# nlp_spancat=spacy.load("spancat_model")
config = {
    "threshold": 0.5,
    "spans_key": "labeled_spans",
    "max_positive": None,
    "model": DEFAULT_SPANCAT_MODEL,
    "suggester": {"@misc": "spacy.ngram_suggester.v1", "sizes": [1, 2, 3]},
}

if 'spancat' not in nlp.pipe_names:
    spancat=nlp.create_pipe('spancat')
    nlp.add_pipe('spancat',config=config)



In [33]:
for _,annotations in train_data:
    for ent in annotations['entities']:
        spancat.add_label(ent[2])
        
other_pipes=[pipe for pipe in nlp.pipe_names if pipe !='spancat']

In [ ]:
# nlp.pipe_names
spancat.labels 

In [ ]:
from numpy import random
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    for batch in spacy.util.minibatch(train_data,size=2):
        random.shuffle(train_data)
        index=0
        losses={}

        for txt,annotation in batch:
            doc =nlp.make_doc(txt)        
            example = Example.from_dict(doc,annotation)
            spancat.initialize()
            
            try:
                losses=spancat.update(
                [example],
                sgd=optimizer,
                drop=0.2)
                print(losses)
            except Exception as e:
                print(e)